In [ ]:
import numpy as np 
import h5py 
from pyFAI import azimuthalIntegrator

import scipy.constants as constants 
import matplotlib.pyplot as plt 

import extra_geom 

import sys 

# Elementary constants 
e_p = constants.elementary_charge
h_p = constants.Planck 
c_l = constants.speed_of_light 

In [ ]:
run_num = 139 

run_file = f'scratch3046/vds/proc/r0{run_num}_proc.cxi' 
with h5py.File(run_file,mode='r') as handle: 
    print(handle['entry_1']['data_1'].keys()) 
    
    data = handle['entry_1']['data_1']['data']
    sel_pulse = int(input(f'Select a pulse out of {data.shape[0]} pulses:'))
    td = np.array(data[sel_pulse,:,:,:]) 
    
mask_file = 'gpfs_3046/usr/Shared/tong/xfel3046/mask/newer_mask.h5'
with h5py.File(mask_file,'r') as f: 
    c_mask = np.asarray(f['combined_mask']) 

In [ ]:
pixel_size = 200e-6 
detector_distance = 0.285
phot_eV = 8000 # beam energy in eV 
phot_J = phot_eV * e_p # beam energy in J 
wavelength = (h_p * c_l) / (phot_J) # beam wavelength in m 

# Number 
n_rad = 200

geom = extra_geom.AGIPD_1MGeometry.from_crystfel_geom('gpfs_3046/usr/Shared/tong/xfel3046/geometry/agipd_september_2022_v03.geom') 
ai = azimuthalIntegrator.AzimuthalIntegrator(dist = detector_distance, detector = geom.to_pyfai_detector(),  wavelength = wavelength) 

assem_td, c_td = geom.position_modules(td) 
assem_msk, c_msk = geom.position_modules(c_mask) 

Qint, Iint = ai.integrate1d(td.reshape(16*512,128), 
                               n_rad,
                               mask = c_mask.reshape(16*512,128),
                               radial_range = (0.0, 2.55),
                               correctSolidAngle = True,
                               polarization_factor = 1, 
                               unit="q_A^-1",
                               method=("BBox","histogram","cython"))

I, _ , _ = ai.integrate2d(td.reshape(16*512,128), 
                               n_rad, 
                               npt_azim = 360,
                               mask = c_mask.reshape(16*512,128),
                               radial_range = (0.0, 2.55), 
                               correctSolidAngle = True, 
                               polarization_factor = 1, 
                               unit="q_A^-1", 
                               method=("BBox","histogram","cython")) 

Ivar = I.std(axis=0) ** 2 

plt.figure(1,dpi=120)
plt.plot(Qint,Iint,'r--')
plt.xlim([0,2.55])
plt.xlabel('$q^{-1}\\ (\mathrm{\AA}^{-1})$')
plt.title(f'Radial average r_{run_num} - p_{sel_pulse}') 

plt.figure(2,dpi=120)
plt.plot(Qint, Ivar,'b--') 
plt.xlim([0,2.55])
plt.xlabel('$q^{-1}\\ (\mathrm{\AA}^{-1})$')
plt.title(f'Radial variance r_{run_num} - p_{sel_pulse}'); 

In [ ]:
# Randomly select and plot 10 simulated diffraction patterns 
fig_handle = plt.figure(3,constrained_layout = True,dpi=200) 
fig_handle.patch.set_facecolor(f'white') 

spec_handle = fig_handle.add_gridspec(nrows = 1 , ncols = 2) 
spec_handle.update(hspace=0.1,wspace=0.6)

ax_1 = fig_handle.add_subplot(spec_handle[0]) 
im_1 = plt.imshow(assem_td,interpolation=None, cmap='plasma')
c_bar_1 = plt.colorbar(im_1,ax=ax_1,ticklocation = 'top',orientation='horizontal') 
ax_1.set_title(f'Pulse {sel_pulse}',fontsize=7) ; 

ax_2 = fig_handle.add_subplot(spec_handle[1]) 
im_2 = plt.imshow(assem_msk,interpolation=None, cmap='gray') 
c_bar_2 = plt.colorbar(im_2,ax=ax_2,ticklocation = 'top',orientation='horizontal')
c_bar_2.set_ticks([0,1]) 
ax_2.set_title('Mask',fontsize=7);  